# Autism Questions

This notebook uses a vector database to find posts in the autism_parenting subreddit that are similar to a giving question.
It seems to give better results than putting the same question into a search on Reddit

In [10]:
import praw
import reddit_credentials

SUBREDDIT_NAME = "autism_parenting"

reddit = praw.Reddit(
    client_id=reddit_credentials.REDDIT_CLIENT_ID,
    client_secret=reddit_credentials.REDDIT_CLIENT_SECRET,
    user_agent=reddit_credentials.REDDIT_USER_AGENT
)

subreddit = reddit.subreddit(SUBREDDIT_NAME)

# Fetch most recent 100 posts
posts = []

for submission in subreddit.new(limit=1000):
    posts.append({
        "title": submission.title,
        "text": submission.selftext,   # empty string for link posts
        "url": submission.url,
        "permalink": f"https://www.reddit.com{submission.permalink}",
        "created_utc": submission.created_utc,
        "score": submission.score,
        "num_comments": submission.num_comments
    })

for i, post in enumerate(posts[:5], start=1):
    print(f"\nPost {i}")
    print("Title:", post["title"])
    print("Text:", post["text"][:200], "...")
    print("URL:", post["url"])



Post 1
Title: Diagnosis mixed emotions
Text: My son got diagnosed today . The neurologist said that she thinks hes lvl 2 ? Does anyone know what this means ? She kind of went back and forth from lvl 2 & 3 but nvr really gave a lvl . She wouldnt  ...
URL: https://www.reddit.com/r/Autism_Parenting/comments/1qx8i7a/diagnosis_mixed_emotions/

Post 2
Title: Autism is not a blessing.
Text: Why do people say Autism is a blessing? I understand that a lot of people on the spectrum have incredible skills and talents but, that also comes with meltdowns and outbursts that are so hard to contr ...
URL: https://www.reddit.com/r/Autism_Parenting/comments/1qx7kru/autism_is_not_a_blessing/

Post 3
Title: When does it get better?
Text: My 5 year old has level 2 autism and dmdd. She was doing well. Helping students, saying more words/sentences. As well as using her visual board. Until she had to go back to school from winter break. J ...
URL: https://www.reddit.com/r/Autism_Parenting/comments/1qx6up4/wh

In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


documents = [p['text'] for p in posts]
model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = model.encode(documents, convert_to_numpy=True)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Inner Product = cosine similarity (after normalization)
index.add(embeddings)

print(f"Indexed {index.ntotal} documents.")


Indexed 999 documents.


In [13]:
def similarity_search(query, k=10):
    # Embed the query
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)

    # Search
    scores, indices = index.search(query_embedding, k)

    # Return results
    results = []
    for score, idx in zip(scores[0], indices[0]):
        results.append({
            "idx": idx,
            "score": float(score),
            "title": posts[idx]['title'],
            "text": posts[idx]['text'],
            "url": posts[idx]['url'],
        })
    return results

query = "How do you deal with kids refusing to eat?"
results = similarity_search(query)

for r in results:
    print(f"\n\n***{r['score']:.3f} → {r['title']}")
    print(r['text'][:200])
    print(r['url'])




***0.712 → 3yo has started refusing meals
My 3 year old has always been somewhat fussy with food but has always been a good eater. He would previously eat as his staple meals:

Roast chicken dinner

Cottage pie

Beef stew

Bolognese 

Soup

B
https://www.reddit.com/r/Autism_Parenting/comments/1qvw0bc/3yo_has_started_refusing_meals/


***0.636 → Behaviors on vacation
Hi all. Newly 4yo, lvl 2. We’ve been on a short vacation for his birthday and he’s refused to eat almost everything we’ve offered to him. He won’t even drink his nutritional shakes (he has to drink 2 
https://www.reddit.com/r/Autism_Parenting/comments/1qsiah0/behaviors_on_vacation/


***0.588 → Seeking Advice: Limited Safe Foods in Autistic Child
I’m hoping to hear from other parents who’ve been through this 🤍
My autistic daughter is 4 and currently only has two safe foods. She is non-verbal, and I’m finding myself constantly anxious about wha
https://www.reddit.com/r/Autism_Parenting/comments/1qgpmcn/seeking_advice_limit